In [3]:
from project_functions_final3 import *
import pandas as pd
import geopandas as gpd
import numpy as np
import os 
from pathlib import Path  
import matplotlib.pyplot as plt

# Variable Assignments

In [4]:
clienttoken = "MLY|6271859919532158|486bcdece7a665eb89a5803f2994b4c9"
bounds = [(12.436781, 55.545432), (12.436781, 55.731815), (12.696966, 55.731815), (12.696966, 55.545432)]
my_shapefile_path = "koebenhavn.shp"
my_shp = gpd.read_file(my_shapefile_path)

In [5]:
ids_file = pd.read_csv('ids.csv')
IDs =  ids_file['image_id'].to_list()
np.random.seed(42)
IDs = list(np.random.choice(IDs, 20000))

In [6]:
strava_csv = pd.read_csv("/home/s214604/data_from_laura/all_edges_monthly_jan_2022-dec_2022_ped/7f42d69dd7bd00b04081b9cbd0d05ab4e5f87353dcd7364c34b8e7c74d42109d-1676891223740.csv")
df_activity = StravaToDF(my_shp, bounds, strava_csv)

# Create dataset

### Get Detections for Mapillary

In [7]:
df_detections, unique_occurences, dict_image = GetDetections(IDs, clienttoken, data_type = 'int')

Loading…: 100%|████████████████████| 20000/20000 [1:19:30<00:00,  4.19it/s]


### Export Detections Dataframe

In [8]:
current_path = os.getcwd()
filepath = Path(current_path+'/detections.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_detections.to_csv(filepath) 

### Create Model Data

In [9]:
pip = GetPointsInPoly(my_shp, clienttoken, bounds, IDs)
model_data = DFTransform(df_detections, pip, df_activity, dict_image, threshold = 0.05)

Loading…: 100%|██████████████████████| 20000/20000 [36:42<00:00,  9.08it/s]
/home/s214604/project_functions_final3.py:102: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_polygons["geometry"] = df_polygons["geometry"].buffer(buffer, cap_style = 2)


### Reassign Activity to Classes

In [10]:
ReassignActivity(model_data)

### Export Model Data

In [11]:
current_path = os.getcwd()
filepath = Path(current_path+'/modeldata_for_parameters.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
model_data.to_csv(filepath)

# Plots


In [ ]:
#PlotShapefile(my_shp, bounds)
#PlotAssignedImages(my_shp, bounds, clienttoken, IDs, buffer_size = 0.0001)
#PlotUnassignedPoints(my_shp, clienttoken, bounds, IDs)
#PlotFractionImages(unique_occurences)
#PlotAvgDetection(dict_image)